In [3]:
library(tensorflow)
library(tfdatasets)
library(dplyr)
library(glue)
library(data.table)
library(ggplot2)
library(keras)
library(tfprobability)
library(stringr)
source('utils.R')

print(tf$version$VERSION)
print(tfp_version() ) ## version checks

[1] ‘0.9’
[1] "2.1.0"


In [5]:
## load data Channel Dataset-----------------------------------------------------
load(file = 'df_paths.rdata',verbose = T)

encoded_size <- 2L  ## number of dims ? Bivariate
ev_shape <-  1L
batch_size = 32

latent_dim <- 2L
n_gru = 7
ds = train_paths[ str_detect(str_c(V1,V2,V3,V4,V5),'1.*(5|7)')==target]
print(table(ds$target))

ft_spec <- ds %>%
  select(-customer_id,-path_no,-path,) %>%
  feature_spec(target ~ .) %>%
  step_categorical_column_with_vocabulary_list(starts_with("V"),vocabulary_list = list('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8')) %>%
  step_embedding_column(starts_with("V"), dimension = function(vocab_size) as.integer(sqrt(vocab_size) + 1)   ## dim=4
  ) %>%
  fit()

ft_spec$dense_features() 

Loading objects:
  df_paths
  train_paths
  test_paths


In [ ]:
## The Models
encoder_model_ts <- function(name = NULL,ngru=n_gru) {
  keras_model_custom(name = name, function(self) {
    self$dense_features <- layer_dense_features(feature_columns=ft_spec$dense_features()) ## ft_spec
    self$reshape = layer_reshape(target_shape= list(5,4)) 
    self$gru1  =   (layer=layer_gru(units = ngru, dropout=0.2 ,recurrent_dropout = 0.1,return_sequences =TRUE) )
    self$gru2  =   (layer=layer_gru(units = ngru, dropout=0.2 ,recurrent_dropout = 0.1,return_sequences =FALSE) )
    self$dense <- layer_dense(units = 2 * latent_dim)
    function (inputs, mask = NULL) {
      x <- inputs[[1]]  ## x, the input, is of size (batch_size, max_length_input) or timesteps Tx
      dense_out = x %>%  
        self$dense_features()  %>% # shape=(batch, col*embed)
        self$reshape() %>% 
        self$gru1(initial_state = inputs[[2]] ) %>%  
        self$gru2(initial_state = inputs[[2]] ) %>%   ## shape: 100/5/5 and 100/5
        self$dense() 
         tfd_multivariate_normal_diag(loc = dense_out[, 1:encoded_size],
                                      scale_diag = tf$nn$softplus(dense_out[, (encoded_size + 1):(2 * encoded_size)] + 1e-5))  
    }
  })
}

decoder_model_ts <- function(name = NULL) {
  keras_model_custom(name = name, function(self) {
    self$dense1 <- layer_dense(units = 7 * 7 , activation = "relu") ## 
    self$dense2 <- layer_dense(units = 1,activation = "sigmoid")  ## 
    function (x, mask = NULL) {
    x =  x %>%
        self$dense1() %>%
        self$dense2() 
      # tfd_independent(tfd_bernoulli(logits = x),reinterpreted_batch_ndims = 1L)
    x
    }
  })
}
compute_kl_loss <- function(latent_prior, approx_posterior, approx_posterior_sample) {
  kl_div <- approx_posterior$log_prob(approx_posterior_sample) -  latent_prior$log_prob(approx_posterior_sample)
  avg_kl_div <- tf$reduce_mean(kl_div)
  abs(avg_kl_div)
}

# Learnable Prior -------------------------------------------------------------------
mixture_components <- 4L
learnable_prior_model <-  function(name = NULL, latent_dim, mixture_components) {
  keras_model_custom(name = name, function(self) {
    self$loc <-   tf$Variable( initial_value=tf$zeros(shape = c(mixture_components, latent_dim)),  name = "loc",   shape = list(mixture_components, latent_dim),   dtype = tf$float32 )
    self$raw_scale_diag <- tf$Variable( initial_value= tf$zeros(shape = c(mixture_components, latent_dim)),  name = "raw_scale_diag",     shape = c(mixture_components, latent_dim), dtype = tf$float32 )
    self$mixture_logits <-  tf$Variable( initial_value=tf$zeros(shape = c(mixture_components)), name = "mixture_logits",    shape = c(mixture_components),  dtype = tf$float32 )
    
    function (x, mask = NULL) {
      tfd_mixture_same_family(
        components_distribution = tfd_multivariate_normal_diag( loc = self$loc,   scale_diag = tf$nn$softplus(self$raw_scale_diag) ),
        mixture_distribution = tfd_categorical(logits = self$mixture_logits)
      )
      # Note how self$loc, self$raw_scale_diag and self$mixture_logits are TensorFlow Variables and thus, persistent and updatable by backprop.
    }
  })
}



In [ ]:
## Training Loop -----------------------------------------------------------
print('training loop..')
optimizer <- tf$optimizers$Adam()
decoder_ts <- decoder_model_ts()
encoder_ts <- encoder_model_ts()
latent_prior_model <- learnable_prior_model(latent_dim = latent_dim, mixture_components = mixture_components)

encoder_init_hidden <- k_zeros(c(batch_size, n_gru), dtype='float32') 

ch_dataset = tensor_slices_dataset(ds) %>%   dataset_batch(batch_size = batch_size,drop_remainder=T) 
num_epochs <- 10
wt_crossentropy_loss = 3
wt_kl_loss = 1
batches_num <-  nrow(ds) / batch_size
num_samples = 10000

for (epoch in seq_len(num_epochs)) {
  iter <-   make_iterator_one_shot(ch_dataset)
  total_loss_kl <- total_loss_nll <- total_loss <- 0
  
  until_out_of_range({
    x <-  iterator_get_next(iter)
    with(tf$GradientTape(persistent = TRUE) %as% tape, {
      approx_posterior = encoder_ts( list(x,encoder_init_hidden))
      approx_posterior_sample <-   approx_posterior %>% tfd_sample(num_samples)  %>% k_mean(axis = 1)
      preds <- decoder_ts(approx_posterior_sample) 
      nll <-   wt_crossentropy_loss*tf$nn$sigmoid_cross_entropy_with_logits(logits = preds, labels = tf$reshape(x$target,shape = preds$shape) )
      latent_prior <- latent_prior_model(NULL) 
      kl_loss <-  wt_kl_loss* compute_kl_loss(  latent_prior,  approx_posterior,  approx_posterior_sample ) 
      loss <- (kl_loss + nll)
    }) ## batch over
    encoder_gradients <- tape$gradient(loss, encoder_ts$variables)
    decoder_gradients <- tape$gradient(loss, decoder_ts$variables)
    
    optimizer$apply_gradients(  purrr::transpose(list(encoder_gradients, encoder_ts$variables))   )
    optimizer$apply_gradients(  purrr::transpose(list(decoder_gradients, decoder_ts$variables))   )
    total_loss <- total_loss + loss
    total_loss_nll <- total_loss_nll + nll
    total_loss_kl <- total_loss_kl + kl_loss
  })
  print(
    glue(
      "Losses (epoch): {epoch}:",
      "  {mean((as.numeric(total_loss_nll))) %>% round(2)} nll",
      "  {(as.numeric(total_loss_kl)) %>% round(2)} kl",
      "  {mean((as.numeric(total_loss))) %>% round(2)} total"
    )
  )
  if (epoch %% 5 == 0) show_space(epoch,size = 15000,dataset=ch_dataset,encoder = encoder_ts)
}  ## train loop 

encoder_ts %>% save_model_tf(filepath = "models/encoder_ts")
## test space -----------------------------------------------------

show_space('test',size = 2000,encoder = encoder_ts, pre = '_lp',
           dataset=tensor_slices_dataset(test_paths) %>%   dataset_batch(batch_size = batch_size,drop_remainder=T) )

